In [27]:
import numpy as np
import pandas as pd
# import project_env as pe
import requests
import logging
import time
# import googlemaps # Make sure to install: $ pip install -U googlemaps 
from datetime import timedelta, datetime
import os

In [28]:
# Show all columns when displaying datasets in notebook
from IPython.display import display
pd.options.display.max_columns = None

## Read and merge source files

Here we first read the source files and merge them.

In [29]:
path = "noda_cleaned"
os.listdir(path)

['Ada.csv',
 'Areg.csv',
 'Bdsr.csv',
 'Bond.csv',
 'Caze.csv',
 'Cgcd.csv',
 'Chrg.csv',
 'Dfdn.csv',
 'Disp.csv',
 'Dsum.csv',
 'Event.csv',
 'Judge.csv',
 'Parm.csv',
 'Prcn.csv',
 'Sent.csv',
 'Wdxr.csv']

In [30]:
csv_ls = os.listdir(path)
dfnames = [csv.split('.')[0] for csv in csv_ls]
df_dict = dict.fromkeys(dfnames, None)

for csv in csv_ls:
    df = pd.read_csv(os.path.join(path, csv), index_col=False)
    df_name = csv.split('.')[0]
    df_dict[df_name] = df

C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


KeyboardInterrupt: 

### Modify Bond & Chrg & Wdxr Table

In [ ]:
# Choose the latest bond for each charge
# AREG_SEQ_NBR was 1 for all records, meaning there were only 1 arrest for each 'sys_nbr - bofi_nbr' combination
idx = df_dict['Bond'].groupby(['SYS_NBR', 'BOFI_NBR', 'CHRG_SEQ_NBR'])['BOND_SEQ_NBR'].transform(max) == df_dict['Bond']['BOND_SEQ_NBR']
df_dict['Bond'] = df_dict['Bond'][idx]
df_dict['Bond']

In [ ]:
# Choose the lead charge for each 'sys_nbr - bofi_nbr' pair
# Some pairs have LEAD_CHARGE_FLAG = N for all charges. In this case, we choose the first row by .first()
df_dict['Chrg'] = df_dict['Chrg'].sort_values('LEAD_CHARGE_FLAG', ascending=False).groupby(['SYS_NBR', 'BOFI_NBR']).first()
df_dict['Chrg'].columns = ['DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'CHARGE_DATE', 'LEAD_CHARGE_FLAG', \
                           'SUB_CHARGE_CODE', 'COUNTS', 'CHRG_ADA_CODE', 'CHRG_JUDGE_CODE', 'BOND_MADE_AMOUNT', \
                           'BOND_SET_AMOUNT', 'BOND_MADE_FLAG']

In [ ]:
# Filter out WITN_IS_VICTIM_FLAG == N
# Ignore witness type (for now)
df_dict['Wdxr'] = df_dict['Wdxr'][df_dict['Wdxr']['WITN_IS_VICTIM_FLAG']=='Y'].drop(columns=['WITN_PTR', 'WITN_TYPE'])

### Start Merging

In [ ]:
######################################
# Merge all data into one data frame #
######################################

# Remove duplicates from Dfdn
df_dict['Dfdn'] = df_dict['Dfdn'].groupby(['BOFI_NBR']).first()
# Merge SENT and DFDN
data_merged = pd.merge(df_dict['Sent'], df_dict['Dfdn'], on='BOFI_NBR', how='left')
data_merged.drop(columns=['CCN_NBR', 'ZIP_DFDN'], inplace=True)
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Caze'], on='SYS_NBR', how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Chrg'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Cgcd'], left_on='CHARGE_CODE_SENT', right_on='CHARGE_CODE', how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Disp'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'DISP_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

In [ ]:
# One sentencing can have multiple conditions -> thus the number or rows increased
data_merged = pd.merge(data_merged, df_dict['Prcn'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                         'DISP_SEQ_NBR', 'SENT_SEQ_NBR'], how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Bond'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'BOFI_NBR', 'SUB_CHARGE_CODE'], how='left')
data_merged

In [ ]:
# multiple bond surety agency can be involced in one bond -> the number of rows increased
data_merged = pd.merge(data_merged, df_dict['Bdsr'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOND_SEQ_NBR'], how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Judge'], left_on='JUDGE_CODE_SENT', right_on='JUDGE_CODE', how='left')
data_merged

In [ ]:
df_dict['Dsum'] = df_dict['Dsum'][df_dict['Dsum']['SCREEN_ADA_CODE'].notna() & df_dict['Dsum']['TRIAL_ADA_CODE'].notna() \
                                  & df_dict['Dsum']['TRIAL_DISP_CODE'].notna()]
df_dict['Dsum'] = df_dict['Dsum'].drop_duplicates()
data_merged = pd.merge(data_merged, df_dict['Dsum'], on=['SYS_NBR', 'BOFI_NBR'], how='left')
data_merged

In [ ]:
df_dict['Screen_Ada'] = df_dict['Ada']
df_dict['Screen_Ada'].columns = ['SCREEN_ADA_CODE', 'ADA_NAME_SCREEN_ADA', 'DOB_SCREEN_ADA', \
                                 'RACE_SCREEN_ADA', 'SEX_SCREEN_ADA', 'PARTY_SCREEN_ADA']
data_merged = pd.merge(data_merged, df_dict['Ada'], left_on='SCREEN_ADA_CODE', right_on='SCREEN_ADA_CODE', how='left')
data_merged

In [ ]:
df_dict['Trial_Ada'] = df_dict['Ada']
df_dict['Trial_Ada'].columns = ['TRIAL_ADA_CODE', 'ADA_NAME_TRIAL_ADA', 'DOB_TRIAL_ADA', \
                                'RACE_TRIAL_ADA', 'SEX_TRIAL_ADA', 'PARTY_TRIAL_ADA']
data_merged = pd.merge(data_merged, df_dict['Ada'], on='TRIAL_ADA_CODE', how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Event'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                          'DISP_SEQ_NBR', 'SENT_SEQ_NBR', 'BOND_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged.drop(columns=['ADA_CODE_EVENT'], inplace=True)
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Parm'], left_on='EVENT_CODE_EVENT', right_on='EVENT_CODE_PARM', how='left')
data_merged

In [ ]:
data_merged = pd.merge(data_merged, df_dict['Wdxr'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

In [ ]:
for name in data_merged.columns:
    if name.endswith('_x'):
        print(name)

In [ ]:
for name in data_merged.columns:
    print(name)